In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
# Carregando os dados
resultados = pd.read_csv('resultados_modelo.csv')

/tmp/ipykernel_107734/727491634.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  resultados = pd.read_csv('resultados_modelo.csv')


In [4]:
falhas = pd.read_csv('falhas_modelo.csv')

In [5]:
status = pd.read_csv('status_modelo.csv')

In [6]:
# Exibir as primeiras linhas de cada dataframe para verificar o carregamento correto
resultados.head()

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA,day,month,year,hour,minute,second
0,2023-5076008,255,718,13,7,80,0.007334,2024-02-02 08:03:03,2.0,2.0,2024.0,8.0,3.0,3.0
1,2023-5076008,255,718,13,7,80,0.007341,2024-02-02 07:37:59,2.0,2.0,2024.0,7.0,37.0,59.0
2,2023-5076008,255,718,10,7,80,0.007357,2024-02-02 08:32:14,2.0,2.0,2024.0,8.0,32.0,14.0
3,2023-5076015,255,718,13,7,80,0.007322,2024-02-07 14:05:16,7.0,2.0,2024.0,14.0,5.0,16.0
4,2023-5076015,255,718,10,7,80,0.007352,2024-02-07 14:31:41,7.0,2.0,2024.0,14.0,31.0,41.0


In [7]:
resultados = resultados.drop('DATA', axis=1)

In [8]:
# Convertendo as colunas de data e hora para um único datetime
resultados['DATA'] = pd.to_datetime(resultados[['day', 'month', 'year', 'hour', 'minute', 'second']])

In [9]:
resultados = resultados.drop(['day', 'month', 'year', 'hour', 'minute', 'second'], axis=1)

In [10]:
falhas.head()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,day,month,year,hour,minute,second
0,2023-2056234,0,2,3,11,4013334,26,paralama dianteiro 01. amassado,20,5,2024,21,31,51
1,2023-2056234,0,2,3,11,4013334,26,porta traseira 04. caroço / pico,20,5,2024,21,30,22
2,2023-2056234,0,2,3,73,2058570,22,módulo do acc (13) peça incorreta ¨,20,5,2024,20,4,55
3,2023-2056234,0,2,3,85,2087715,8,tampa traseira 05. sujeira na pintura,11,4,2024,12,15,39
4,2023-3896001,0,0,0,6,2043971,20,tampa dianteira 05. sujeira na pintura,5,6,2024,17,38,51


In [11]:
falhas['DATA'] = pd.to_datetime(falhas[['day', 'month', 'year', 'hour', 'minute', 'second']])

In [12]:
falhas = falhas.drop(['day', 'month', 'year', 'hour', 'minute', 'second'], axis=1)

In [13]:
status.head()

,KNR,STATUS,day,month,year,hour,minute,second
0,2023-2016173,6,6,6,2023,7,50,27
1,2023-2016173,78,6,6,2023,7,50,27
2,2023-2026032,6,20,6,2023,17,35,51
3,2023-2026032,78,20,6,2023,17,35,51
4,2023-2026085,6,22,6,2023,20,22,36


In [14]:
status['DATA'] = pd.to_datetime(status[['day', 'month', 'year', 'hour', 'minute', 'second']])

In [15]:
status = status.drop(['day', 'month', 'year', 'hour', 'minute', 'second'], axis=1)

In [16]:
resultados['KNR'] = resultados['KNR'].astype(str)
falhas['KNR'] = falhas['KNR'].astype(str)
status['KNR'] = status['KNR'].astype(str)

In [27]:
# Mesclando os dados com base em 'KNR' e 'datetime'
merged_data = pd.merge(resultados, falhas, on=['KNR', 'DATA'], how='outer')
merged_data = pd.merge(merged_data, status, on=['KNR', 'DATA'], how='outer')

In [28]:
# Verificando a fusão
merged_data.head()

,KNR,NAME,ID,STATUS_x,UNIT,VALUE_ID,VALUE,DATA,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,STATUS_y
0,2023-0276001,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-14 15:12:03,1.0,0.0,3.0,60.0,4004400,20.0,a - início retrabalho aberto (a),NaN
1,2023-0276001,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-30 15:29:40,1.0,0.0,3.0,87.0,FHQWRAP,21.0,teste eletrônico (ibn_3) nok,NaN
2,2023-0276001,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-01 20:27:26,1.0,0.0,3.0,11.0,2078090,25.0,porta traseira 04. caroço / pico,NaN
3,2023-0276001,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-01 20:30:19,1.0,0.0,3.0,11.0,2078090,25.0,painel lateral sujeira na pintura,NaN
4,2023-0276001,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-01 20:41:16,1.0,0.0,3.0,10.0,4004717,25.0,régua da moldura do teto solar desencaixado (&),NaN


In [29]:
# Preenchendo valores ausentes para colunas numéricas com a média
numerical_cols = merged_data.select_dtypes(include=['float64', 'int64']).columns
merged_data[numerical_cols] = merged_data[numerical_cols].fillna(merged_data[numerical_cols].mean())

# Preenchendo valores ausentes para colunas categóricas com o valor mais frequente (modo)
categorical_cols = merged_data.select_dtypes(include=['object']).columns
merged_data[categorical_cols] = merged_data[categorical_cols].fillna(merged_data[categorical_cols].mode().iloc[0])

In [30]:
merged_data

,KNR,NAME,ID,STATUS_x,UNIT,VALUE_ID,VALUE,DATA,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,STATUS_y
0,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-11-14 15:12:03,1.0,0.0,3.0,60.0,4004400,20.0,a - início retrabalho aberto (a),52.284936
1,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-11-30 15:29:40,1.0,0.0,3.0,87.0,FHQWRAP,21.0,teste eletrônico (ibn_3) nok,52.284936
2,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:27:26,1.0,0.0,3.0,11.0,2078090,25.0,porta traseira 04. caroço / pico,52.284936
3,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:30:19,1.0,0.0,3.0,11.0,2078090,25.0,painel lateral sujeira na pintura,52.284936
4,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:41:16,1.0,0.0,3.0,10.0,4004717,25.0,régua da moldura do teto solar desencaixado (&),52.284936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833291,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-25 21:00:16,1.0,0.0,3.0,85.0,2059053,8.0,porta traseira 01. amassado,52.284936
7833292,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-25 21:00:27,1.0,0.0,3.0,85.0,2059053,8.0,porta traseira risco,52.284936
7833293,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-26 07:59:47,1.0,0.0,3.0,76.0,4004411,16.0,porta traseira risco,52.284936
7833294,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-26 14:36:33,1.0,0.0,3.0,86.0,FHQWRAP,27.0,smart inspection peça nok,52.284936


In [32]:
label_encoder = LabelEncoder()
merged_data['FALHA'] = label_encoder.fit_transform(merged_data['FALHA'])

In [33]:
merged_data

,KNR,NAME,ID,STATUS_x,UNIT,VALUE_ID,VALUE,DATA,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,STATUS_y,FALHA_encoded
0,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-11-14 15:12:03,1.0,0.0,3.0,60.0,4004400,20.0,34,52.284936,34
1,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-11-30 15:29:40,1.0,0.0,3.0,87.0,FHQWRAP,21.0,3892,52.284936,3892
2,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:27:26,1.0,0.0,3.0,11.0,2078090,25.0,2817,52.284936,2817
3,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:30:19,1.0,0.0,3.0,11.0,2078090,25.0,2291,52.284936,2291
4,2023-0276001,125.907805,38.303426,10.014899,4.612078,1,0.01005,2023-12-01 20:41:16,1.0,0.0,3.0,10.0,4004717,25.0,3312,52.284936,3312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833291,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-25 21:00:16,1.0,0.0,3.0,85.0,2059053,8.0,2814,52.284936,2814
7833292,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-25 21:00:27,1.0,0.0,3.0,85.0,2059053,8.0,2884,52.284936,2884
7833293,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-26 07:59:47,1.0,0.0,3.0,76.0,4004411,16.0,2884,52.284936,2884
7833294,2024-5076015,125.907805,38.303426,10.014899,4.612078,1,0.01005,2024-01-26 14:36:33,1.0,0.0,3.0,86.0,FHQWRAP,27.0,3506,52.284936,3506


In [34]:
features = merged_data.drop(['KNR', 'NAME', 'FALHA', 'ID', 'STATUS_x', 'UNIT', 'VALUE_ID', 'VALUE', 'DATA', 'MODELO', 'COR', 'MOTOR', 'ESTACAO', 'USUARIO', 'HALLE', 'STATUS_y'], axis=1)
target = merged_data['FALHA']

In [35]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

In [39]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [40]:
class CarDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Criando datasets e dataloaders
train_dataset = CarDataset(X_train, y_train)
test_dataset = CarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [41]:
import torch.nn as nn
import torch.optim as optim

class CarFailurePredictor(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CarFailurePredictor, self).__init__()
        self.layer_1 = nn.Linear(input_size, 128)
        self.layer_2 = nn.Linear(128, 64)
        self.layer_3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.layer_1(x))
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.dropout(x)
        x = self.layer_3(x)
        return x

# Definindo o modelo, critério de perda e otimizador
input_size = X_train.shape[1]
num_classes = len(np.unique(y_train))
model = CarFailurePredictor(input_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [47]:
# Função de treinamento
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

In [48]:
# Treinando o modelo novamente
train_model(model, train_loader, criterion, optimizer)

IndexError: Target 4161 is out of bounds.